In [3]:
import pandas as pd

# --- Load data ---
proj_rank_df = pd.read_csv("project_preferences.csv")   # project columns, rows are ranked packages
priority_df = pd.read_csv("our_rankings.csv")           # columns: Project, amy and dom's ranking

# --- Build project preferences ---
project_prefs = {}
for project in proj_rank_df.columns:
    prefs = proj_rank_df[project].dropna().tolist()
    project_prefs[project] = prefs

# --- Build package preferences ---
# Mapping: project → rank number
project_priority = priority_df.set_index("Project Title")["Ranking"].to_dict()

# Every package uses the same ranking of projects (lower number = more preferred)
packages = set(sum(project_prefs.values(), []))
package_prefs = {
    pkg: sorted(project_priority.keys(), key=lambda proj: project_priority[proj])
    for pkg in packages
}

# --- Gale-Shapley algorithm ---
free_projects = list(project_prefs.keys())
current_matches = {}  # {package: project}
next_proposal_index = {proj: 0 for proj in project_prefs}

while free_projects:
    proj = free_projects.pop(0)
    if next_proposal_index[proj] >= len(project_prefs[proj]):
        # This project has proposed to all packages → skip
        continue

    # Get next package it hasn’t proposed to
    pkg = project_prefs[proj][next_proposal_index[proj]]
    next_proposal_index[proj] += 1

    if pkg not in current_matches:
        # Package is free → match
        current_matches[pkg] = proj
    else:
        current_proj = current_matches[pkg]
        # Package decides between current and new proposer
        if package_prefs[pkg].index(proj) < package_prefs[pkg].index(current_proj):
            # Prefers new proposer
            current_matches[pkg] = proj
            free_projects.append(current_proj)
        else:
            # Reject new proposer
            free_projects.append(proj)

# --- Result ---
print(current_matches)



{'P10': 'Eviction Debt Assistance Fund', 'P12': 'Reentry Housing Support', 'P16': 'Rainy-Day Fund for Greater Equity', 'P13': 'The Pearson Project: Kitchen Renovation', 'P3': "Black Wall Street Hub for Amplifying Durham's Entrepreneur Community", 'P2': "DCI's ConnectEd to Career Pipeline", 'P1': 'Necessity-Driven Entrepreneurial Support', 'P7': 'Durham: The City of Books', 'P15': 'Homeowner & Homebuyer Initiated Program (HHIP)', 'P4': 'Transitional & Independent Housing', 'P6': 'Green Stormwater Infrastructure Installations', 'P18': 'Bridging Business Opportunity and Equity', 'P17': 'Durham Affordable Sustainable Homes (DASH)', 'P11': 'Merrick-Moore Community Preservation Project', 'P8': 'Digital Literacy Training and Computers', 'P14': 'Multifamily Acquisition Support & Supply (MASS)', 'P9': 'Durham Peer Support Reentry Partnership', 'P19': 'Home and Neighborhood Preservation Initiative', 'P20': 'Community Infrastructure Investment for Equitable Engagement', 'P5': 'Community-Rooted™ A